In [1]:
import netCDF4 as nc
import rasterio
from rasterio.transform import from_origin
from rasterio.enums import Resampling

def convert_netcdf_to_geotiff(input_file, output_folder):
    # Open the NetCDF file
    nc_dataset = nc.Dataset(input_file, 'r')

    for variable_name in nc_dataset.variables:
        # Skip dimensions and auxiliary variables
        if variable_name not in nc_dataset.dimensions:
            # Extract variable data
            variable_data = nc_dataset.variables[variable_name][:]

            # Define GeoTIFF output file path
            output_file = f"{output_folder}/{variable_name}.tif"

            # Get the geospatial information from the NetCDF file
            lon = nc_dataset.variables['lon'][:]
            lat = nc_dataset.variables['lat'][:]
            transform = from_origin(lon.min(), lat.max(), abs(lon[1] - lon[0]), abs(lat[1] - lat[0]))

            # Write GeoTIFF file
            with rasterio.open(output_file, 'w', driver='GTiff',
                               height=variable_data.shape[0], width=variable_data.shape[1],
                               count=1, dtype=str(variable_data.dtype), crs='EPSG:4326',
                               transform=transform) as dst:
                dst.write(variable_data, 1)

    # Close the NetCDF file
    nc_dataset.close()

if __name__ == "__main__":
    # Specify the input NetCDF file and output folder
    input_nc_file = 'a.nc'
    output_folder = 'output_geotiffs'

    # Create the output folder if it doesn't exist
    import os
    os.makedirs(output_folder, exist_ok=True)

    # Convert NetCDF to GeoTIFF
    convert_netcdf_to_geotiff(input_nc_file, output_folder)


FileNotFoundError: [Errno 2] No such file or directory: 'a.nc'